# Preprocessing
---

In [1]:
import pandas as pd
import re
import sqlite3

In [2]:
press_list = ['daum', 'naver',
              'seoulilbo', 'dtoday', 'asiailbo', 'labortoday', 'm-i',
              'ekn', 'busan', 'imaeil', 'kookje', 'yeongnam']
press_dict = {'daum':'다음', 'naver':'네이버',
              'seoulilbo':'서울일보', 'dtoday':'일간투데이', 'asiailbo':'아시아일보',
              'labortoday':'매일노동뉴스', 'm-i':'매일일보', 'ekn':'에너지경제',
              'busan':'부산일보', 'imaeil':'매일신문', 'kookje':'국제신문',
              'yeongnam':'영남일보'}

daum_press_list = ['EBS', 'IT동아', 'JTBC', 'KBS', 'KTV',
                   'MBC', 'MBN', 'SBS', 'SBSCNBC', 'YTN',
                   '지디넷코리아', 'bnt뉴스', '게임동아', '게임톡', '경향신문',
                   '국민일보', '기자협회보', '노컷뉴스', '뉴스1', '뉴시스',
                   '데일리안', '동아사이언스', '동아일보', '디지털타임스', '로이터',
                   '매경게임진', '매일경제', '머니S', '머니투데이', '문화일보',
                   '미디어오늘', '서울경제', '서울신문', '세계일보', '아시아경제',
                   '아이뉴스24', '연합뉴스', '연합뉴스TV', '오마이뉴스', '오토타임즈',
                   '이데일리', '전자신문', '조선비즈', '조선일보', '중앙일보',
                   '채널A', '코리아헤럴드', '쿠키뉴스', '파이낸셜뉴스', '포토친구',
                   '프레시안', '한겨레', '한국경제', '한국경제TV', '한국일보',
                   '헤럴드경제']
naver_press_list = ['JTBC', 'KBS', 'MBC', 'MBN', 'SBS',
                    'SBS CNBC', 'TV조선', 'YTN', 'ZDNet Korea', '강원일보',
                    '경향신문', '국민일보', '기자협회보', '노컷뉴스', '뉴스1',
                    '뉴시스', '데일리안', '동아사이언스', '동아일보', '디지털데일리',
                    '디지털타임스', '로이터', '매일경제', '매일신문', '머니S',
                    '머니투데이', '문화일보', '미디어오늘', '부산일보', '블로터',
                    '서울경제', '서울신문', '세계일보', '아시아경제', '아이뉴스24',
                    '여성신문', '연합뉴스', '연합뉴스TV', '오마이뉴스', '이데일리',
                    '일다', '전자신문', '조선비즈', '조선일보', '조세일보',
                    '중앙일보', '참세상', '채널A', '코리아헤럴드', '코메디닷컴',
                    '파이낸셜뉴스', '프레시안', '한겨레', '한국경제', '한국경제TV',
                    '한국일보', '헤럴드경제', '헬스조선']
daum_naver_press_list = tuple(set(daum_press_list + naver_press_list))

section_list = ['society', 'politics', 'economic', 'culture', 'digital', 'global']
section_dict = {'society':'사회', 'politics':'정치', 'economic':'경제',
               'culture':'문화', 'digital':'IT', 'global':'세계'}

base_urls = {'daum':
             {'society':'http://media.daum.net/breakingnews/society',
              'politics':'http://media.daum.net/breakingnews/politics',
              'economic':'http://media.daum.net/breakingnews/economic',
              'culture':'http://media.daum.net/breakingnews/culture',
              'digital':'http://media.daum.net/breakingnews/digital',
              'global':'http://media.daum.net/breakingnews/foreign'
             },
             'seoulilbo':
             {'society':'http://www.seoulilbo.com/news/articleList.html?sc_section_code=S1N10&view_type=sm',
              'politics':'http://www.seoulilbo.com/news/articleList.html?sc_section_code=S1N8&view_type=sm',
              'economic':'http://www.seoulilbo.com/news/articleList.html?sc_section_code=S1N9&view_type=sm',
              'culture':'http://www.seoulilbo.com/news/articleList.html?sc_section_code=S1N11&view_type=sm',
              'digital':'',
              'global':''
             },
             'dtoday':
             {'society':'',
              'politics':'http://www.dtoday.co.kr/news/articleList.html?sc_section_code=S1N1&view_type=sm',
              'economic':'http://www.dtoday.co.kr/news/articleList.html?sc_section_code=S1N2&view_type=sm',
              'culture':'',
              'digital':'',
              'global':''
             },
             'asiailbo':
             {'society':'http://www.asiailbo.co.kr/etnews/?cid=21030000',
              'politics':'http://www.asiailbo.co.kr/etnews/?cid=21010000',
              'economic':'http://www.asiailbo.co.kr/etnews/?cid=21020000',
              'culture':'http://www.asiailbo.co.kr/etnews/?cid=21040000',
              'digital':'',
              'global':''
             },
             'labortoday':
             {'society':'http://www.labortoday.co.kr/news/articleList.html?sc_section_code=S1N3&view_type=sm',
              'politics':'http://www.labortoday.co.kr/news/articleList.html?sc_section_code=S1N2&view_type=sm',
              'economic':'',    # 정치, 경제
              'culture':'',
              'digital':'',
              'global':''
             },
             'm-i':
             {'society':'http://www.m-i.kr/news/articleList.html?sc_section_code=S1N3&view_type=sm',
              'politics':'http://www.m-i.kr/news/articleList.html?sc_section_code=S1N1&view_type=tm',
              'economic':'http://www.m-i.kr/news/articleList.html?sc_section_code=S1N2&view_type=sm',
              'culture':'http://www.m-i.kr/news/articleList.html?sc_section_code=S1N22&view_type=tm',
              'digital':'',
              'global':''
             },
             'ekn':
             {'society':'http://www.ekn.kr/news/section_list_all.html?sec_no=25',
              'politics':'',    # 정치, 사회
              'economic':'http://www.ekn.kr/news/section_list_all.html?sec_no=130',
              'culture':'',
              'digital':'',
              'global':''
             },
             'busan':
             {'society':'http://news20.busan.com/news/social.jsp',
              'politics':'http://news20.busan.com/news/politics.jsp',
              'economic':'http://news20.busan.com/EconomyAndOcean/econocean.jsp',
              'culture':'http://news20.busan.com/news/culture.jsp',
              'digital':'',
              'global':''
             },
             'imaeil':
             {'society':'http://news.imaeil.com/SocietyAll/',
              'politics':'http://news.imaeil.com/PoliticsAll/',
              'economic':'http://news.imaeil.com/EconomyAll/',
              'culture':'http://news.imaeil.com/CultureAll/',
              'digital':'',
              'global':'http://news.imaeil.com/InternationalAll/'
             },
             'kookje':
             {'society':'http://www.kookje.co.kr/sub.htm?code=0300&vHeadTitle=%BB%E7%C8%B8',
              'politics':'http://www.kookje.co.kr/sub.htm?code=0100&vHeadTitle=%C1%A4%C4%A1',
              'economic':'http://www.kookje.co.kr/sub.htm?code=0200&vHeadTitle=%B0%E6%C1%A6',
              'culture':'http://www.kookje.co.kr/sub.htm?code=0500&vHeadTitle=%B9%AE%C8%AD',
              'digital':'http://www.kookje.co.kr/sub.htm?code=0800&vHeadTitle=IT%B0%FA%C7%D0',
              'global':'http://www.kookje.co.kr/sub.htm?code=0400&vHeadTitle=%B1%B9%C1%A6'
             },
             'yeongnam':
             {'society':'http://www.yeongnam.com/mnews/newsview.do?mode=subMain&cId=04',
              'politics':'http://www.yeongnam.com/mnews/newsview.do?mode=subMain&cId=02',
              'economic':'http://www.yeongnam.com/mnews/newsview.do?mode=subMain&cId=03',
              'culture':'http://www.yeongnam.com/mnews/newsview.do?mode=subMain&cId=08',
              'digital':'',    # 교육, 과학
              'global':'http://www.yeongnam.com/mnews/newsview.do?mode=subMain&cId=06'
             }
            }

headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"}

In [3]:
# 기본 정규식
reg_ex = {'email': r'[a-zA-Z0-9.!#$%&\'*+/=?^_`{|}~-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+',
          'sb': r'\[[^\]]*\]',
          'pb': r'\<[^\>]*\>'}

# 매칭되면 기사를 삭제하는 태그, 단어 및 정규식
stop_title_sb = ['[포토]', '[프로필]', '[★화보]', '[게시판]', '[경향포토]',
                 '[리빙포인트]', '[머니S포토]', '[부고]', '[부음]', '[사람 人 사람]',
                 '[사진]', '[서울포토]', '[영어]', "[오래전 '이날']", '[오마이포토]',
                 '[이 시각 코스피]', '[이 시각 코스닥]', '[인사]', '[코스닥 공시]', '[코스닥(개장)]',
                 '[코스닥(마감)]', '[코스피(개장)]', '[코스피(마감)]', '[포토뉴스]', '[표]',
                 '[한경로보뉴스]', '[영상]', '[오늘의 국회 ', '[오늘의 주요일정]']

stop_title_pb = ['<부고>', '<오늘의 조간 정치뉴스>', '<인사>', '<포토>', '<코>', '<유>']

stop_content_reg_ex = [r'^동영상[\s]*뉴스']

# 매칭되면 해당 내용만 삭제하는 태그, 단어 및 정규식
stop_content_reg_ex_crop = [reg_ex['email']]

stop_content_reg_ex_ml_crop = [r'^[\s]*\【.*?\】([^=\n]*기자[\s]*=)?',
                               r'^[\s]*\[.*?\]([^=\n)]*기자[\s]*=)?',
                               r'^[\s]*\(.*?\)([^=\n]*기자[\s]*=)?',
                               r'^[ㄱ-ㅎㅏ-ㅣ가-힣]+[\s]?기자[\s]?',
                               r'^© News1.*\n',
                               '(- Copyrights )?ⓒ[^\.]*$',
                               r'\【.*?\】[\s]*$',
                               r'\[.*?\][\s]*$',
                               r'\(.*?\)[\s]*$',
                               r'[\s\.=][ㄱ-ㅎㅏ-ㅣ가-힣]+[\s]?기자$']

# [r'^[\s]*\【[^\】]*\】([^=\n]*기자[\s]*=)?',
# r'^[\s]*\[[^\]]*\]([^=\n)]*기자[\s]*=)?',
# r'^[\s]*\([^\)]*\)([^=\n]*기자[\s]*=)?',
# r'^[ㄱ-ㅎㅏ-ㅣ가-힣]+[\s]?기자[\s]?',
# r'\【[^\【]*\】[\s]*$',
# r'\[[^\[]*\][\s]*$',
# r'\([^\(]*\)[\s]*$',
# r'[\s\.][ㄱ-ㅎㅏ-ㅣ가-힣]+[\s]?기자$']
# r'^[\s]*\【[^\】]*=[^\】]*\】([^=\n]*기자[\s]*=)?'
# r'^[\s]*\[[^\]]*=[^\]]*\]([^=\n)]*기자[\s]*=)?'
# r'^[\s]*\([^\)]*=[^\)]*\)([^=\n]*기자[\s]*=)?'
# /.+\n
# r'^[]{0}[]$'.format(press_ko)'

In [4]:
# # Delete
# conn = sqlite3.connect('db/daum.db')
# cur = conn.cursor()

# cur.execute("DELETE FROM daum WHERE sections LIKE '세계'")

# conn.commit()

In [5]:
# # Rename column
# conn = sqlite3.connect('db/daum.db')
# cur = conn.cursor()

# cur.execute("ALTER TABLE daum RENAME TO daum_temp")

# cur.execute("CREATE TABLE daum(a_ids TEXT primary key, \
#             dates DATE, times TIME, titles TEXT, contents TEXT, \
#             press TEXT, authors TEXT, sections TEXT, urls TEXT)")

# cur.execute("INSERT INTO daum (a_ids, dates, times, titles, contents, press, authors, sections, urls) \
#             SELECT a_ids, dates, times, titles, contents, press_ko, authors, section_ko, urls \
#             FROM daum_temp")

# conn.commit()

## Null Data 제거
---

* date, time, title, content, press column이 NULL 또는 빈 문자열(공백문자로만 이뤄진 경우 포함)인 경우

In [6]:
def delete_null_data(db_name, table_name):
    conn = sqlite3.connect('db/' + db_name + '.db')
    cur = conn.cursor()
    
    cols = ('date', 'time', 'title', 'content', 'press')
    for col in cols:
        cur.execute("DELETE FROM {0} WHERE {1} IS NULL OR TRIM({1}) = ''".format(table_name, col))

    conn.commit()

In [7]:
# delete_null_data('daum', 'daum')

In [8]:
# delete_null_data('naver', 'naver')

In [9]:
delete_null_data('news_db_new', 'Article')

## Daum, Naver에서 리스트에 없는 언론사 제외
---

In [10]:
def delete_excluded_press(db_name, table_name, press_list):
    conn = sqlite3.connect('db/' + db_name + '.db')
    cur = conn.cursor()
    
    cur.execute("DELETE FROM {0} WHERE press NOT IN {1}".format(table_name, tuple(press_list)))

    conn.commit()

In [11]:
# delete_excluded_press('daum', 'daum', daum_press_list)

In [12]:
# delete_excluded_press('naver', 'naver', naver_press_list)

In [13]:
delete_excluded_press('news_db_new', 'Article', daum_naver_press_list)

## Contents 길이가 100 미만일 경우 제거
---

In [14]:
# Contents 길이 분석
def analysis_contents_length(db_name, table_name):
    conn = sqlite3.connect('db/' + db_name + '.db')
    cur = conn.cursor()
    
    cur.execute("SELECT content FROM " + table_name)

    temp_content_list = []
    for content_tuple in cur.fetchall():
        temp_content_list.append(content_tuple[0])

    df = pd.DataFrame({'content':temp_content_list})
    
    df['content_length'] = None
    for idx, content in enumerate(df.content):
        df.content_length.iloc[idx] = len(content)
    
    return df

In [15]:
# df = analysis_contents_length('daum', 'daum')

# df.head()    

In [16]:
# df.info()

In [17]:
# df.describe(percentiles=(0.01, 0.02, 0.03, 0.04, 0.05, 0.1))

In [18]:
# df = analysis_contents_length('naver', 'naver')

# df.head()

In [19]:
# df.info()

In [20]:
# df.describe(percentiles=(0.01, 0.02, 0.03, 0.04, 0.05, 0.1))

In [21]:
def delete_short_contents(db_name, table_name, threshold_len):
    conn = sqlite3.connect('db/' + db_name + '.db')
    cur = conn.cursor()
    
    cur.execute("DELETE FROM {0} WHERE LENGTH(content) < {1}".format(table_name, threshold_len))

    conn.commit()

In [22]:
# delete_short_contents('daum', 'daum', 100)

# df = analysis_contents_length('daum', 'daum')

# df.describe()

In [23]:
# delete_short_contents('naver', 'naver', 100)

# df = analysis_contents_length('naver', 'naver')

# df.describe()

In [24]:
delete_short_contents('news_db_new', 'Article', 100)

## Contents가 같을 경우 제거
---

In [25]:
# conn = sqlite3.connect('db/daum.db')
# cur = conn.cursor()

# cur.execute("SELECT DISTINCT contents FROM daum")

# len(cur.fetchall())

In [26]:
# conn = sqlite3.connect('db/naver.db')
# cur = conn.cursor()

# cur.execute("SELECT DISTINCT contents FROM naver")

# len(cur.fetchall())

In [27]:
def delete_dupl_contents(db_name, table_name):
    conn = sqlite3.connect('db/' + db_name + '.db')
    cur = conn.cursor()
    
    cur.execute("DELETE FROM {0} WHERE a_id NOT IN (SELECT min(a_id) FROM {0} GROUP BY content)"
                .format(table_name))

    conn.commit()

In [28]:
# delete_dupl_contents('daum', 'daum')

In [29]:
# delete_dupl_contents('naver', 'naver')

In [30]:
delete_dupl_contents('news_db_new', 'Article')